In [2]:
import pandas as pd
import requests
import os
import numpy as np
import json

# 收集

In [3]:
# twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv','r', encoding = False)
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv', encoding = "iso-8859-1")
# twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv', sep=r'(?:,|\s+)', engine='python')

In [4]:
## 获取文件推特图片预测 image_predictions
### 1.确定image_predictions 的下载网址链接
image_predictions_url = 'https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv'
### 2.下载和保存文件
response = requests.get(image_predictions_url)
with open('image_predictions.csv', mode = 'wb') as file:
    file.write(response.content)
image_predictions = pd.read_csv('image_predictions.csv', sep = '\t')

In [5]:
## 获取文件3 每条推特的数据
tweet_json = pd.read_json('tweet_json.txt',lines = True)

# 评估

##### 质量
###### `WeRateDogs 推特档案(twitter_archive_enhanced)`的评估
- timestamp, retweeted_status_timestamp 应当调整为时间序列
- tweet_id 应当为字符串str
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id 采用 fillna 来将其转化为int
- rating_numerator分数最大值为1776，不合理
- rating_denominator分数最大值为170，不合理，这些评级通常以 10 作为分母
- tweet_id‘832215909146226688’的rating_denominator应当为9.75，而不是75
- 缺少了推特的网址链接 expanded_urls
- name为'a', 'an', 'the'都是原始数据中缺失值，应当替换replace为'None'
- 转发的推相当于对于某条狗的重复评价，应当删除该行数据。

###### `image_predictions` 的评估
- tweet_id 应当为字符串str
- 缺少图片推测（twitter中为2356个，图片推测仅为2075个）

###### `tweet_json` 的评估
- possibly_sensitive、possibly_sensitive_appealable 应当为布尔型bool
- 缺少api获取成果（twitter档案中为2356个，api获取成果仅为2352个）
- 有效性：in_reply_to_status_id、in_reply_to_user_id、retweeted_status_id、retweeted_status_user_id 应当为字符串str
- id和id_str 重复，删除str列
- possibly_sensitive和possibly_sensitive_appealable 重复，删除possibly_sensitive_appealable列
- extended_entities、possibly_sensitive 缺少数据


##### 整洁度
###### `WeRateDogs 推特档案(twitter_archive_enhanced)`的评估
- tweet_json中的转发数（retweet_count）和喜欢数（favorite_count）应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
- 神经网络推测出的狗的品种 (或其他物体、动物等），即可信度最高的image_predictions中的p1，应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
`twitter_archive_enhanced 的评估`

In [7]:
twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [57]:
twitter_archive_enhanced.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [74]:
# 验证是否有重复项,结果为没有
twitter_archive_enhanced['tweet_id'].duplicated().sum()

0

In [44]:
# 完整度
twitter_archive_enhanced[twitter_archive_enhanced['expanded_urls'].isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
185,856330835276025856,NaN,NaN,2017-04-24 02:15:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Jenna_Marbles: @dog_rates Thanks for ratin...,8.563302e+17,66699013.0,2017-04-24 02:13:14 +0000,NaN,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None


In [6]:
# 查找出 转发的tweet，结果为有，共181条转发推。
twitter_archive_enhanced[twitter_archive_enhanced['retweeted_status_id'].notnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 19 to 2260
Data columns (total 17 columns):
tweet_id                      181 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     181 non-null object
source                        181 non-null object
text                          181 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 180 non-null object
rating_numerator              181 non-null int64
rating_denominator            181 non-null int64
name                          181 non-null object
doggo                         181 non-null object
floofer                       181 non-null object
pupper                        181 non-null object
puppo                         181 non-null object
dtypes: float64(4), int64(3), object(10)
memor

In [87]:
# 找出狗名字的重复数据
twitter_archive_enhanced[twitter_archive_enhanced['name'].duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
12,889665388333682689,NaN,NaN,2017-07-25 01:55:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo that seems to be on the fence a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889665388...,13,10,None,None,None,None,puppo
23,887473957103951883,NaN,NaN,2017-07-19 00:47:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Canela. She attempted some fancy porch...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
24,887343217045368832,NaN,NaN,2017-07-18 16:08:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",You may not have known you needed to see this ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887343217...,13,10,None,None,None,None,None
25,887101392804085760,NaN,NaN,2017-07-18 00:07:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This... is a Jubilant Antarctic House Bear. We...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887101392...,12,10,None,None,None,None,None
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,19607400.0,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
35,885518971528720385,NaN,NaN,2017-07-13 15:19:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I have a new hero and his name is Howard. 14/1...,NaN,NaN,NaN,https://twitter.com/4bonds2carbon/status/88551...,14,10,None,None,None,None,None
37,885167619883638784,NaN,NaN,2017-07-12 16:03:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a corgi undercover as a malamute....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/885167619...,13,10,None,None,None,None,None
41,884441805382717440,NaN,NaN,2017-07-10 15:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I present to you, Pup in Hat. Pup in Hat is gr...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/884441805...,14,10,None,None,None,None,None


In [93]:
# name为'a', 'an', 'the'都是原始数据中缺失值，因为并没有狗的名字会叫做冠词。
twitter_archive_enhanced['name'].value_counts()

None           745
a               55
Charlie         12
Lucy            11
Oliver          11
Cooper          11
Penny           10
Lola            10
Tucker          10
Winston          9
Bo               9
the              8
Sadie            8
Daisy            7
Bailey           7
Buddy            7
Toby             7
an               7
Bella            6
Rusty            6
Leo              6
Koda             6
Scout            6
Jack             6
Jax              6
Milo             6
Oscar            6
Stanley          6
Dave             6
very             5
              ... 
Pherb            1
Lilli            1
Mark             1
Ricky            1
Toffee           1
Harlso           1
Vinnie           1
Sunshine         1
Vinscent         1
Rodney           1
Nida             1
Tassy            1
Tater            1
this             1
Coleman          1
Kobe             1
Oddie            1
Clarq            1
Binky            1
Einstein         1
Sobe             1
Edmund      

In [162]:
# rating_numerator的统计
twitter_archive_enhanced['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [163]:
# rating_denominator的统计
# 评级通常以 10 作为分母，所以结果中不是10的均应当
twitter_archive_enhanced['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [180]:
# 不用
# 找到评价基准分数大于10的，存入test_0227
# twitter_archive_enhanced_clean
# test_0227 = twitter_archive_enhanced_clean[(np.abs(twitter_archive_enhanced_clean['rating_denominator']) > 10).any(1)]
### test_0227 = twitter_archive_enhanced_clean[(np.abs(twitter_archive_enhanced_clean.rating_denominator) <= 10)]

In [182]:
# 不用
### test_0227['rating_numerator'].value_counts()

12      558
11      463
10      461
13      351
9       156
8       102
14       54
7        54
5        37
6        32
3        19
4        15
2         9
1         9
75        2
0         2
420       2
15        2
26        1
24        1
182       1
27        1
960       1
1776      1
17        1
666       1
Name: rating_numerator, dtype: int64

`image_predictions 的评估`

In [49]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [56]:
image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [84]:
# 每条推的照片数量，结果发现最多4张，最少1张
image_predictions['img_num'].sort_values()

In [73]:
# 验证是否有重复项，结果为没有重复
image_predictions['tweet_id'].duplicated().sum()

0

`tweet_json 的评估`

In [51]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2352 entries, 0 to 2351
Data columns (total 31 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2352 non-null datetime64[ns]
display_text_range               2352 non-null object
entities                         2352 non-null object
extended_entities                2073 non-null object
favorite_count                   2352 non-null int64
favorited                        2352 non-null bool
full_text                        2352 non-null object
geo                              0 non-null float64
id                               2352 non-null int64
id_str                           2352 non-null int64
in_reply_to_screen_name          78 non-null object
in_reply_to_status_id            78 non-null float64
in_reply_to_status_id_str        78 non-null float64
in_reply_to_user_id              78 non-null float64
in_reply_to_user_id_str          78 n

In [7]:
# 从tweet_json中提取三个关键列'id','retweet_count','favorite_count',根据这三个指标建立tweet_keys
tweet_info_keys = ['id','retweet_count','favorite_count']
tweet_keys = pd.DataFrame(tweet_json, columns=tweet_info_keys)

In [14]:
tweet_keys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 3 columns):
id                2352 non-null int64
retweet_count     2352 non-null int64
favorite_count    2352 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [69]:
tweet_keys.sample(10)

,id,retweet_count,favorite_count
1320,706169069255446529,2497,4269
44,883482846933004288,10336,46741
1357,703079050210877440,3480,8044
1549,688908934925697024,869,2304
1693,681231109724700672,539,2616
1452,695314793360662529,1664,3991
492,813172488309972993,2227,10344
2337,666094000022159362,78,168
1748,678969228704284672,527,1788
1530,689993469801164801,516,1610


In [70]:
tweet_keys.describe()

,id,retweet_count,favorite_count
count,2.352000e+03,2352.000000,2352.000000
mean,7.425913e+17,3134.932398,8109.198980
std,6.846210e+16,5237.846296,11980.795669
min,6.660209e+17,0.000000,0.000000
25%,6.783949e+17,618.000000,1417.000000
50%,7.193536e+17,1456.500000,3596.500000
75%,7.991219e+17,3628.750000,10118.000000
max,8.924206e+17,79116.000000,132318.000000


In [72]:
# 验证是否有重复项，结果为没有重复
tweet_keys['id'].duplicated().sum()

0

In [59]:
tweet_json.describe()

,contributors,coordinates,favorite_count,geo,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,possibly_sensitive,possibly_sensitive_appealable,quoted_status_id,quoted_status_id_str,retweet_count
count,0.0,0.0,2352.000000,0.0,2.352000e+03,2.352000e+03,7.800000e+01,7.800000e+01,7.800000e+01,7.800000e+01,2211.0,2211.0,2.900000e+01,2.900000e+01,2352.000000
mean,NaN,NaN,8109.198980,NaN,7.425913e+17,7.425913e+17,7.455079e+17,7.455079e+17,2.014171e+16,2.014171e+16,0.0,0.0,8.162686e+17,8.162686e+17,3134.932398
std,NaN,NaN,11980.795669,NaN,6.846210e+16,6.846210e+16,7.582492e+16,7.582492e+16,1.252797e+17,1.252797e+17,0.0,0.0,6.164161e+16,6.164161e+16,5237.846296
min,NaN,NaN,0.000000,NaN,6.660209e+17,6.660209e+17,6.658147e+17,6.658147e+17,1.185634e+07,1.185634e+07,0.0,0.0,6.721083e+17,6.721083e+17,0.000000
25%,NaN,NaN,1417.000000,NaN,6.783949e+17,6.783949e+17,6.757419e+17,6.757419e+17,3.086374e+08,3.086374e+08,0.0,0.0,7.888183e+17,7.888183e+17,618.000000
50%,NaN,NaN,3596.500000,NaN,7.193536e+17,7.193536e+17,7.038708e+17,7.038708e+17,4.196984e+09,4.196984e+09,0.0,0.0,8.340867e+17,8.340867e+17,1456.500000
75%,NaN,NaN,10118.000000,NaN,7.991219e+17,7.991219e+17,8.257804e+17,8.257804e+17,4.196984e+09,4.196984e+09,0.0,0.0,8.664587e+17,8.664587e+17,3628.750000
max,NaN,NaN,132318.000000,NaN,8.924206e+17,8.924206e+17,8.862664e+17,8.862664e+17,8.405479e+17,8.405479e+17,0.0,0.0,8.860534e+17,8.860534e+17,79116.000000


In [161]:
# 查看tweet.json数据
tweet_json.sample(5)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,possibly_sensitive_appealable,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,truncated,user
1250,NaN,NaN,2016-03-18 02:46:49,"[0, 108]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 710658682619617280, 'id_str'...",2513,False,Here's a brigade of puppers. All look very pre...,NaN,...,0.0,NaN,NaN,NaN,633,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
599,NaN,NaN,2016-11-15 21:49:12,"[0, 103]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 670444949847023616, 'id_str'...",0,False,RT @dog_rates: This is Paull. He just stubbed ...,NaN,...,0.0,NaN,NaN,NaN,2142,False,{'created_at': 'Sat Nov 28 03:31:48 +0000 2015...,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1982,NaN,NaN,2015-12-04 21:05:23,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 672884420823527424, 'id_str'...",1653,False,Marvelous dog here. Rad ears. Not very soft. L...,NaN,...,0.0,NaN,NaN,NaN,895,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2350,NaN,NaN,2015-11-15 23:05:30,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666029276303482880, 'id_str'...",132,False,This is a western brown Mitsubishi terrier. Up...,NaN,...,0.0,NaN,NaN,NaN,48,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2226,NaN,NaN,2015-11-22 01:20:44,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 668237637640183808, 'id_str'...",6573,False,This is Torque. He served his nickel. Better n...,NaN,...,0.0,NaN,NaN,NaN,3083,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


In [110]:
# 完整度
tweet_json[tweet_json['extended_entities'].isnull()]

In [11]:
# tweet_json['truncated'].duplicated()
# tweet_json['truncated'].duplicated().sum()
tweet_json['user']

# 清理

### 保存原始数据的副本

In [75]:
twitter_archive_enhanced_clean = twitter_archive_enhanced.copy()
image_predictions_clean = image_predictions.copy()
tweet_json_clean = tweet_json.copy()
tweet_keys_clean = tweet_keys.copy()

### 整洁度
#### tweet_json中的转发数（retweet_count）和喜欢数（favorite_count）应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
##### 定义
将tweet_json中的retweet_count 和favorite_count导入一个 DataFrame 中，并它通过tweet_id拼接到 twitter_archive_enhanced_clean中。

##### 代码

In [76]:
# tweet_keys_clean中的id列改名为tweet_id后，twitter_archive_enhanced_clean就可以和tweet_keys_clean拼接。
tweet_keys_clean.rename(columns={'id':'tweet_id'}, inplace = True)

# 拼接
twitter_archive_enhanced_clean = pd.merge(twitter_archive_enhanced_clean, tweet_keys_clean, on='tweet_id', how='left')

##### 测试

In [ ]:
tweet_keys_clean.info()

In [77]:
twitter_archive_enhanced_clean[twitter_archive_enhanced_clean['retweet_count'].isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None,NaN,NaN
118,869988702071779329,NaN,NaN,2017-05-31 18:47:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: We only rate dogs. This is quit...,8.591970e+17,4.196984e+09,2017-05-02 00:04:57 +0000,https://twitter.com/dog_rates/status/859196978...,12,10,quite,None,None,None,None,NaN,NaN
155,861769973181624320,NaN,NaN,2017-05-09 02:29:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: ""Good afternoon class today we'...",8.066291e+17,4.196984e+09,2016-12-07 22:38:52 +0000,https://twitter.com/dog_rates/status/806629075...,13,10,None,None,None,None,None,NaN,NaN
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None,NaN,NaN


In [78]:
# 可发现retweet_count 和favorite_count的index已经加入twitter_archive_enhanced_clean中
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count                 23

#### image_predictions中的p1，应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
##### 定义
通过 image_predictions 中的tweet_id，将其中的p1拼接到twitter_archive_enhanced_clean中。
##### 代码

In [79]:
twitter_archive_enhanced_clean = pd.merge(twitter_archive_enhanced_clean, image_predictions, on='tweet_id', how='left')

##### 测试

In [80]:
# 可发现p1,p1_conf和p1_dog已经加入twitter_archive_enhanced_clean中
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count                 23

### 质量
#### `twitter_archive_enhanced`中name为'a', 'an', 'the'都是原始数据中缺失值，替换为'None'
##### 定义
使用函数，将 a, an, the 转变为 None。

##### 代码

In [14]:
# name的映射
a_clean = {'a': 'None', 'an': 'None', 'the': 'None'}
# 替换名字函数
def replace_name(twitter_archive_enhanced):
    if twitter_archive_enhanced['name'] in a_clean.keys():
        clean_name = a_clean[twitter_archive_enhanced['name']]
        return clean_name
    else:
        return twitter_archive_enhanced['name']

twitter_archive_enhanced_clean['name'] = twitter_archive_enhanced_clean.apply(replace_name, axis=1)

##### 测试

In [16]:
# 'name'为'a', 'an', 'the'均已经替换为'None'
twitter_archive_enhanced_clean['name'].value_counts()

None        815
Charlie      12
Oliver       11
Lucy         11
Cooper       11
Tucker       10
Lola         10
Penny        10
Winston       9
Bo            9
Sadie         8
Daisy         7
Bailey        7
Buddy         7
Toby          7
Koda          6
Bella         6
Milo          6
Oscar         6
Jack          6
Rusty         6
Dave          6
Stanley       6
Scout         6
Leo           6
Jax           6
Sunny         5
Louis         5
George        5
Finn          5
           ... 
Akumi         1
Jonah         1
Skittle       1
Angel         1
Coleman       1
Pippin        1
Tessa         1
Terrenth      1
all           1
Ed            1
Billy         1
Katie         1
Ember         1
River         1
Stuart        1
Ralphus       1
Anna          1
Chaz          1
life          1
Rilo          1
Rambo         1
Tango         1
Tycho         1
Maisey        1
Brat          1
Jomathan      1
Horace        1
Siba          1
Snoop         1
Antony        1
Name: name, Length: 954,

#### 错误的数据类型
##### 定义
- timestamp, retweeted_status_timestamp 中均有‘ +0000’ 的后缀，应当删除，并转化为 datetime 数据类型。
- tweet_id 应当为字符串str
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id 采用 fillna 来将其转化为int

##### 代码

In [81]:
# 删除timestamp的‘ +0000’ 的后缀
twitter_archive_enhanced_clean.timestamp = twitter_archive_enhanced_clean.timestamp.str.strip('\s\+0000')
twitter_archive_enhanced_clean.retweeted_status_timestamp = twitter_archive_enhanced_clean.retweeted_status_timestamp.str.strip('\s\+0000')
# 将timestamp转化为 datetime 类型
twitter_archive_enhanced_clean.timestamp = pd.to_datetime(twitter_archive_enhanced_clean.timestamp)
twitter_archive_enhanced_clean.retweeted_status_timestamp = pd.to_datetime(twitter_archive_enhanced_clean.retweeted_status_timestamp)

In [82]:
# 将tweet_id转为str数据类型
twitter_archive_enhanced_clean.tweet_id = twitter_archive_enhanced_clean.tweet_id.astype('str')
# 回复相关id和转发相关id为float64，且存在NaN，则采用 fillna 来将其转化为int
# 参考 https://stackoverflow.com/questions/47013973/pandas-converts-float64-to-int
twitter_archive_enhanced_clean.in_reply_to_status_id = twitter_archive_enhanced_clean.in_reply_to_status_id.fillna(0).astype(np.int64)
twitter_archive_enhanced_clean.in_reply_to_user_id = twitter_archive_enhanced_clean.in_reply_to_user_id.fillna(0).astype(np.int64)
twitter_archive_enhanced_clean.retweeted_status_id = twitter_archive_enhanced_clean.retweeted_status_id.fillna(0).astype(np.int64)
twitter_archive_enhanced_clean.retweeted_status_user_id = twitter_archive_enhanced_clean.retweeted_status_user_id.fillna(0).astype(np.int64)

##### 测试

In [58]:
twitter_archive_enhanced_clean.retweeted_status_id

In [93]:
# 测试timestamp, retweeted_status_timestamp 中‘ +0000’ 的后缀是否已经删除，并转化为 datetime 数据类型。
twitter_archive_enhanced_clean.sample(30)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,RATING
1300,707693576495472641,0,0,2016-03-09 22:24:31,"<a href=""http://twitter.com/download/iphone"" r...",This is Jiminus. He's in a tub for some reason...,0,0,NaT,https://twitter.com/dog_rates/status/707693576...,...,bathtub,0.499525,False,tub,0.488014,False,washbasin,0.009298,False,0.7
364,829141528400556032,0,0,2017-02-08 01:35:19,"<a href=""http://twitter.com/download/iphone"" r...",This is Malcolm. He goes from sneaky tongue sl...,0,0,NaT,https://twitter.com/dog_rates/status/829141528...,...,golden_retriever,0.573140,True,cocker_spaniel,0.111159,True,gibbon,0.094127,False,1.2
1295,707983188426153984,707980065892667392,2319108198,2016-03-10 17:35:20,"<a href=""http://twitter.com/download/iphone"" r...",@serial @MrRoles OH MY GOD I listened to all o...,0,0,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2
1558,688789766343622656,0,0,2016-01-17 18:27:32,"<a href=""http://twitter.com/download/iphone"" r...",Meet Pubert. He's a Kerplunk Rumplestilt. Cann...,0,0,NaT,https://twitter.com/dog_rates/status/688789766...,...,American_Staffordshire_terrier,0.599660,True,Staffordshire_bullterrier,0.380976,True,bull_mastiff,0.003889,True,0.8
1836,676098748976615425,0,0,2015-12-13 17:57:57,"<a href=""http://twitter.com/download/iphone"" r...",Extremely rare pup here. Very religious. Alway...,0,0,NaT,https://twitter.com/dog_rates/status/676098748...,...,walking_stick,0.162179,False,sandal,0.129086,False,purse,0.081412,False,0.3
893,759446261539934208,0,0,2016-07-30 17:51:13,"<a href=""http://twitter.com/download/iphone"" r...",No no no this is all wrong. The Walmart had to...,0,0,NaT,https://twitter.com/wsaznews/status/7591675587...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
805,772114945936949249,0,0,2016-09-03 16:52:02,"<a href=""http://twitter.com/download/iphone"" r...",This is Finn. He's very nervous for the game. ...,0,0,NaT,https://twitter.com/dog_rates/status/772114945...,...,Chihuahua,0.803293,True,toy_terrier,0.052980,True,Italian_greyhound,0.037239,True,1.0
2138,670040295598354432,0,0,2015-11-27 00:43:49,"<a href=""http://twitter.com/download/iphone"" r...",ððð 10/10 for the dog and the owner h...,0,0,NaT,https://twitter.com/dog_rates/status/670040295...,...,web_site,0.901552,False,borzoi,0.026660,True,Chihuahua,0.012438,True,1.0
1506,691756958957883396,0,0,2016-01-25 22:58:05,"<a href=""http://twitter.com/download/iphone"" r...",THE BRITISH ARE COMING\nTHE BRITISH ARE COMING...,0,0,NaT,https://twitter.com/dog_rates/status/691756958...,...,Saint_Bernard,0.342571,True,boxer,0.289096,True,Pembroke,0.076463,True,1.0
1609,685547936038666240,0,0,2016-01-08 19:45:39,"<a href=""http://twitter.com/download/iphone"" r...",Everybody needs to read this. Jack is our firs...,0,0,NaT,https://twitter.com/dog_rates/status/685547936...,...,web_site,0.923987,False,oscilloscope,0.009712,False,hand-held_computer,0.008769,False,1.4


In [84]:
# 查看 数据类型是否转化为 datetime，str 和 int 类型
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count    

#### 转发的推相当于对于该狗的重复评价，应当删除该行狗的数据。
##### 定义
- 转发的tweet应当删除，找出带有retweeted_status_id的行，然后去除该行
##### 代码

In [32]:
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean[twitter_archive_enhanced_clean['retweeted_status_id'].isnull()]

##### 测试

In [34]:
# retweeted_status_id，retweeted_status_user_id，retweeted_status_timestamp这三列的值已经均为NaN。
twitter_archive_enhanced_clean.sample(30)

#### 原始数据不合理
##### 定义
- rating_numerator分数最大值为1776，不合理。rating_denominator分数最大值为170，不合理，这些评级通常以 10 作为分母。
- 针对以上两个问题确定了新指标RATING，即每条狗的rating_numerator/rating_denominator的比值，并去除NaN值和非常大的奇异值（本项目认为大于2即为奇异值），RATING的高低可以有效体现每条狗的评级，分数越大，表明狗评级越高、越收到欢迎。之后的分析结论也是基于RATING一列。
##### 代码

In [40]:
# rating_numerator_morethan20 = twitter_archive_enhanced_clean[(np.abs(twitter_archive_enhanced_clean.rating_numerator) >= 20)]
# rating_numerator_morethan20

In [85]:
twitter_archive_enhanced_clean['RATING'] = twitter_archive_enhanced_clean['rating_numerator']/twitter_archive_enhanced_clean['rating_denominator']

In [89]:
# 去除 twitter_archive_enhanced_clean['RATING'] 中的NaN值和非常大的奇异值（本项目认为大于20即为奇异值）
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean[np.abs(twitter_archive_enhanced_clean['RATING']) <= 2]

In [ ]:
##### 测试

In [94]:
twitter_archive_enhanced_clean.sample(30)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,RATING
1308,707297311098011648,0,0,2016-03-08 20:09:54,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She can go from sweet to scary...,0,0,NaT,https://twitter.com/dog_rates/status/707297311...,...,Blenheim_spaniel,0.370717,True,Shih-Tzu,0.201566,True,black-footed_ferret,0.101559,False,1.0
1393,700029284593901568,0,0,2016-02-17 18:49:22,"<a href=""http://twitter.com/download/iphone"" r...",This is Coops. His ship is taking on water. So...,0,0,NaT,https://twitter.com/dog_rates/status/700029284...,...,West_Highland_white_terrier,0.726571,True,Maltese_dog,0.176828,True,Dandie_Dinmont,0.070134,True,1.0
319,834477809192075265,0,0,2017-02-22 18:59:48,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Leo. He was a skater pu...,829501995190984704,4196983835,2017-02-09 01:27:41,https://twitter.com/dog_rates/status/829501995...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2
1147,726935089318363137,0,0,2016-05-02 00:43:25,"<a href=""http://twitter.com/download/iphone"" r...",This is Sprout. He's just precious af. 12/10 I...,0,0,NaT,https://twitter.com/dog_rates/status/726935089...,...,teddy,0.821615,False,toy_poodle,0.083749,True,Lakeland_terrier,0.033318,True,1.2
1776,677961670166224897,0,0,2015-12-18 21:20:32,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Izzy. She's showing off the dance move...,0,0,NaT,https://vine.co/v/iKuMDuYV0aZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1
1566,687841446767013888,0,0,2016-01-15 03:39:15,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",13/10 I can't stop watching this (vid by @k8ly...,0,0,NaT,https://vine.co/v/iOWwUPH1hrw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3
520,810254108431155201,0,0,2016-12-17 22:43:27,"<a href=""http://twitter.com/download/iphone"" r...","This is Gus. He likes to be close to you, whic...",0,0,NaT,https://twitter.com/dog_rates/status/810254108...,...,Staffordshire_bullterrier,0.292556,True,American_Staffordshire_terrier,0.261233,True,Border_terrier,0.062375,True,1.2
1491,692894228850999298,0,0,2016-01-29 02:17:12,"<a href=""http://twitter.com/download/iphone"" r...",Meet Tupawc. He's actually a Christian rapper....,0,0,NaT,https://twitter.com/dog_rates/status/692894228...,...,German_short-haired_pointer,0.876977,True,bluetick,0.036615,True,basset,0.017848,True,1.0
2302,667012601033924608,0,0,2015-11-18 16:12:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Klevin. He laughs a lot. Very cool dog...,0,0,NaT,https://twitter.com/dog_rates/status/667012601...,...,hyena,0.987230,False,African_hunting_dog,0.012601,False,coyote,0.000057,False,0.9
1099,735991953473572864,0,0,2016-05-27 00:32:10,"<a href=""http://twitter.com/download/iphone"" r...",This is Maxaroni. He's curly af. Also rather f...,0,0,NaT,https://twitter.com/dog_rates/status/735991953...,...,cocker_spaniel,0.961643,True,toy_poodle,0.011547,True,soft-coated_wheaten_terrier,0.004903,True,1.1


In [92]:
# 查看是否还存在奇异值和NaN。
twitter_archive_enhanced_clean['RATING'].value_counts()

1.200000    562
1.100000    469
1.000000    463
1.300000    351
0.900000    157
0.800000    102
1.400000     54
0.700000     54
0.500000     38
0.600000     32
0.300000     19
0.400000     15
0.200000     11
0.100000      8
0.818182      2
0.000000      2
1.500000      2
1.700000      1
0.733333      1
0.636364      1
1.250000      1
Name: RATING, dtype: int64